# 🫀 Treinamento de Detecção de Picos PPG

Este notebook demonstra o pipeline completo para treinar um modelo de detecção
de picos em sinais PPG, usando o MIMIC-II como base e seus próprios dados para
fine-tuning.

## O que vamos fazer:
1. **Baixar** uma amostra do MIMIC-II (PPG + ECG sincronizados)
2. **Criar os Rótulos** detectando picos R no ECG e transferindo para o PPG
3. **Treinar** um modelo simples (1D CNN) para detectar picos
4. **Testar** no seu dado do ESP32

## 📦 Instalação de Dependências

Execute esta célula apenas uma vez para instalar as bibliotecas necessárias.

In [ ]:
!pip install wfdb numpy scipy scikit-learn matplotlib torch

## 1. Importações e Configurações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, butter, filtfilt, resample
from scipy.ndimage import gaussian_filter1d
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.rcParams['figure.figsize'] = (14, 4)
plt.rcParams['figure.dpi'] = 100

# Parâmetros do pipeline
MIMIC_FS = 125       # Taxa de amostragem do MIMIC-II (Hz)
ESP32_FS = 757       # Taxa de amostragem do seu ESP32 (Hz)
TARGET_FS = 125      # Taxa alvo após decimação (Hz)
WINDOW_SEC = 2       # Tamanho da janela em segundos
WINDOW_SIZE = WINDOW_SEC * TARGET_FS  # 250 amostras por janela

print(f"✅ Configurações carregadas:")
print(f"   - Janela: {WINDOW_SEC}s = {WINDOW_SIZE} amostras @ {TARGET_FS}Hz")

## 2. Carregando Dados do MIMIC-II

O MIMIC-II Waveform Database contém sinais de pacientes de UTI.
Vamos usar a biblioteca `wfdb` para baixar uma amostra.

> **Nota:** O download pode demorar na primeira vez (~50MB por registro).

In [ ]:
import wfdb

# Lista de registros de exemplo do MIMIC-II Matched Subset
# Estes contêm PPG (PLETH) e ECG (II ou V) sincronizados
MIMIC_RECORDS = [
    'mimic3wdb-matched/1.0/p00/p000020/p000020-2183-04-28-17-47',  # Exemplo
]

def download_mimic_sample(record_name, duration_sec=60):
    """
    Baixa uma amostra do MIMIC-II.
    
    Args:
        record_name: Nome do registro no PhysioNet
        duration_sec: Duração em segundos para baixar
        
    Returns:
        ppg: Sinal PPG (PLETH)
        ecg: Sinal ECG
        fs: Taxa de amostragem
    """
    try:
        # Baixar registro
        record = wfdb.rdrecord(
            record_name,
            pn_dir='mimic3wdb-matched/1.0',
            sampfrom=0,
            sampto=duration_sec * MIMIC_FS
        )
        
        # Encontrar canais PPG e ECG
        sig_names = [s.upper() for s in record.sig_name]
        
        ppg_idx = None
        ecg_idx = None
        
        for i, name in enumerate(sig_names):
            if 'PLETH' in name:
                ppg_idx = i
            elif name in ['II', 'I', 'V', 'AVR', 'AVL', 'AVF']:
                ecg_idx = i
                
        if ppg_idx is None or ecg_idx is None:
            raise ValueError(f"Canais não encontrados. Disponíveis: {sig_names}")
            
        ppg = record.p_signal[:, ppg_idx]
        ecg = record.p_signal[:, ecg_idx]
        
        # Remover NaN
        ppg = np.nan_to_num(ppg, nan=np.nanmean(ppg))
        ecg = np.nan_to_num(ecg, nan=np.nanmean(ecg))
        
        return ppg, ecg, record.fs
        
    except Exception as e:
        print(f"⚠️ Erro ao baixar do PhysioNet: {e}")
        print("   Gerando dados sintéticos para demonstração...")
        return generate_synthetic_data(duration_sec, MIMIC_FS)


def generate_synthetic_data(duration_sec, fs):
    """
    Gera dados sintéticos de PPG e ECG para demonstração.
    Útil quando o PhysioNet está offline ou lento.
    """
    t = np.linspace(0, duration_sec, duration_sec * fs)
    
    # Simular batimentos cardíacos (~70 BPM com variabilidade)
    hr_base = 70 / 60  # Hz
    rr_intervals = 1/hr_base + 0.05 * np.sin(2 * np.pi * 0.1 * t)  # RSA simulada
    
    # ECG sintético (ondas R simples)
    ecg = np.zeros_like(t)
    peak_times = []
    current_time = 0
    for i, rr in enumerate(rr_intervals):
        if current_time >= duration_sec:
            break
        peak_idx = int(current_time * fs)
        if peak_idx < len(ecg):
            # Onda R como gaussiana estreita
            ecg += 2.0 * np.exp(-((t - current_time) ** 2) / (0.005 ** 2))
            peak_times.append(current_time)
        current_time += rr_intervals[min(i, len(rr_intervals)-1)]
    
    # Adicionar ruído ao ECG
    ecg += 0.1 * np.random.randn(len(ecg))
    
    # PPG sintético (atrasado ~200ms do ECG)
    ppg = np.zeros_like(t)
    ptt = 0.2  # Pulse Transit Time (segundos)
    for peak_time in peak_times:
        ppg_time = peak_time + ptt
        if ppg_time < duration_sec:
            # Onda PPG como gaussiana mais larga
            ppg += 1.0 * np.exp(-((t - ppg_time) ** 2) / (0.03 ** 2))
            # Adicionar nó dicrótico
            ppg += 0.3 * np.exp(-((t - ppg_time - 0.15) ** 2) / (0.02 ** 2))
    
    # Normalizar e adicionar ruído
    ppg = (ppg - ppg.min()) / (ppg.max() - ppg.min())
    ppg += 0.05 * np.random.randn(len(ppg))
    
    return ppg, ecg, fs

# Tentar baixar do MIMIC, ou usar dados sintéticos
print("📥 Baixando dados do MIMIC-II (ou gerando sintéticos)...")
ppg_raw, ecg_raw, fs = generate_synthetic_data(60, MIMIC_FS)  # Usar sintético por padrão

print(f"✅ Dados carregados: {len(ppg_raw)} amostras @ {fs}Hz ({len(ppg_raw)/fs:.1f}s)")

### Visualização dos Sinais Brutos

In [ ]:
# Plotar primeiros 10 segundos
t = np.arange(len(ppg_raw)) / fs
fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)

axes[0].plot(t[:10*fs], ecg_raw[:10*fs], 'b-', linewidth=0.8)
axes[0].set_ylabel('ECG (mV)')
axes[0].set_title('📊 Sinais MIMIC-II (ou Sintéticos) - Primeiros 10 segundos')
axes[0].grid(True, alpha=0.3)

axes[1].plot(t[:10*fs], ppg_raw[:10*fs], 'r-', linewidth=0.8)
axes[1].set_ylabel('PPG')
axes[1].set_xlabel('Tempo (s)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Criando os Rótulos (Labels)

Esta é a parte **crucial**: detectamos os picos R no ECG (fácil e preciso)
e usamos essa informação para marcar onde estão os picos no PPG.

In [ ]:
def detect_ecg_r_peaks(ecg, fs, min_distance_sec=0.4):
    """
    Detecta picos R no ECG usando um algoritmo simples mas robusto.
    
    Em produção, você usaria Pan-Tompkins ou similar.
    """
    # Filtro passa-banda (5-15 Hz para isolar QRS)
    nyq = fs / 2
    b, a = butter(2, [5/nyq, 15/nyq], btype='band')
    ecg_filt = filtfilt(b, a, ecg)
    
    # Elevar ao quadrado para realçar picos
    ecg_squared = ecg_filt ** 2
    
    # Suavizar
    ecg_smooth = gaussian_filter1d(ecg_squared, sigma=fs*0.02)
    
    # Detectar picos
    min_distance = int(min_distance_sec * fs)
    peaks, properties = find_peaks(
        ecg_smooth,
        distance=min_distance,
        height=np.percentile(ecg_smooth, 70)
    )
    
    return peaks


def transfer_peaks_to_ppg(ecg_peaks, ecg, ppg, fs, ptt_range=(0.15, 0.35)):
    """
    Transfere os picos do ECG para o PPG considerando o Pulse Transit Time (PTT).
    
    O pico sistólico do PPG ocorre ~200-300ms após o pico R do ECG.
    Procuramos o máximo local nessa janela.
    """
    ppg_peaks = []
    
    ptt_min_samples = int(ptt_range[0] * fs)
    ptt_max_samples = int(ptt_range[1] * fs)
    
    for ecg_peak in ecg_peaks:
        # Janela de busca no PPG
        search_start = ecg_peak + ptt_min_samples
        search_end = ecg_peak + ptt_max_samples
        
        if search_end >= len(ppg):
            continue
            
        # Encontrar máximo local nessa janela
        window = ppg[search_start:search_end]
        local_max_idx = np.argmax(window)
        ppg_peak = search_start + local_max_idx
        
        ppg_peaks.append(ppg_peak)
    
    return np.array(ppg_peaks)


# Detectar picos R no ECG
ecg_r_peaks = detect_ecg_r_peaks(ecg_raw, fs)
print(f"🔍 Detectados {len(ecg_r_peaks)} picos R no ECG")

# Transferir para o PPG
ppg_peaks = transfer_peaks_to_ppg(ecg_r_peaks, ecg_raw, ppg_raw, fs)
print(f"🎯 Mapeados {len(ppg_peaks)} picos no PPG")

# Calcular RR intervals para validação
rr_intervals = np.diff(ppg_peaks) / fs * 1000  # em ms
hr_mean = 60000 / np.mean(rr_intervals)
print(f"❤️ HR médio calculado: {hr_mean:.1f} BPM")

### Visualização: ECG vs PPG com Picos

In [ ]:
# Plotar 5 segundos com picos marcados
start_sec = 2
end_sec = 7
start_idx = int(start_sec * fs)
end_idx = int(end_sec * fs)

fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)

# ECG com picos R
axes[0].plot(t[start_idx:end_idx], ecg_raw[start_idx:end_idx], 'b-', linewidth=1)
ecg_peaks_in_range = ecg_r_peaks[(ecg_r_peaks >= start_idx) & (ecg_r_peaks < end_idx)]
axes[0].scatter(t[ecg_peaks_in_range], ecg_raw[ecg_peaks_in_range], 
                c='red', s=100, zorder=5, label='Pico R (ECG)')
axes[0].set_ylabel('ECG')
axes[0].set_title('🎯 Transferência de Picos: ECG → PPG')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# PPG com picos sistólicos
axes[1].plot(t[start_idx:end_idx], ppg_raw[start_idx:end_idx], 'purple', linewidth=1)
ppg_peaks_in_range = ppg_peaks[(ppg_peaks >= start_idx) & (ppg_peaks < end_idx)]
axes[1].scatter(t[ppg_peaks_in_range], ppg_raw[ppg_peaks_in_range], 
                c='green', s=100, zorder=5, label='Pico Sistólico (PPG)')
axes[1].set_ylabel('PPG')
axes[1].set_xlabel('Tempo (s)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Mostrar PTT (linhas conectando)
for ecg_p, ppg_p in zip(ecg_peaks_in_range, ppg_peaks_in_range):
    if ecg_p < end_idx and ppg_p < end_idx:
        axes[1].axvline(x=t[ecg_p], color='red', linestyle='--', alpha=0.3)
        axes[1].annotate('', xy=(t[ppg_p], ppg_raw[ppg_p]), 
                        xytext=(t[ecg_p], ppg_raw[ppg_p]),
                        arrowprops=dict(arrowstyle='->', color='gray', alpha=0.5))

plt.tight_layout()
plt.show()

## 4. Criando o Dataset de Treinamento

Agora vamos criar janelas de PPG com seus respectivos rótulos.
O rótulo é um vetor binário indicando onde está o pico.

In [ ]:
def create_training_windows(ppg, peaks, window_size, stride=None):
    """
    Cria janelas de treinamento com rótulos.
    
    Args:
        ppg: Sinal PPG completo
        peaks: Índices dos picos no sinal
        window_size: Tamanho da janela em amostras
        stride: Passo entre janelas (default: window_size // 2)
        
    Returns:
        X: Array de janelas de PPG (N, window_size)
        y: Array de rótulos binários (N, window_size)
    """
    if stride is None:
        stride = window_size // 2
    
    X = []
    y = []
    
    # Normalização Z-score do sinal completo
    ppg_norm = (ppg - np.mean(ppg)) / np.std(ppg)
    
    # Criar conjunto de picos para busca rápida
    peaks_set = set(peaks)
    
    for start in range(0, len(ppg) - window_size, stride):
        end = start + window_size
        
        # Extrair janela
        window = ppg_norm[start:end]
        
        # Criar rótulo binário (1 onde tem pico, 0 caso contrário)
        label = np.zeros(window_size)
        for i in range(window_size):
            if (start + i) in peaks_set:
                # Marcar região ao redor do pico (±2 amostras = ±16ms @ 125Hz)
                for offset in range(-2, 3):
                    if 0 <= i + offset < window_size:
                        label[i + offset] = 1
        
        # Só incluir janelas com pelo menos 1 pico
        if np.sum(label) > 0:
            X.append(window)
            y.append(label)
    
    return np.array(X), np.array(y)


# Criar dataset
X, y = create_training_windows(ppg_raw, ppg_peaks, WINDOW_SIZE)

print(f"📊 Dataset criado:")
print(f"   - Janelas: {X.shape[0]}")
print(f"   - Tamanho da janela: {X.shape[1]} amostras")
print(f"   - Rótulo: {y.shape[1]} amostras (binário)")

# Dividir em treino/validação
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n🔀 Divisão:")
print(f"   - Treino: {len(X_train)} janelas")
print(f"   - Validação: {len(X_val)} janelas")

### Visualização de uma Janela de Treino

In [ ]:
# Mostrar exemplo de janela com rótulo
fig, axes = plt.subplots(2, 1, figsize=(12, 5), sharex=True)

sample_idx = 5
t_window = np.arange(WINDOW_SIZE) / TARGET_FS

axes[0].plot(t_window, X_train[sample_idx], 'purple', linewidth=1)
axes[0].set_ylabel('PPG (Z-score)')
axes[0].set_title(f'📦 Exemplo de Janela de Treino #{sample_idx}')
axes[0].grid(True, alpha=0.3)

axes[1].fill_between(t_window, y_train[sample_idx], alpha=0.5, color='green', label='Rótulo (onde tem pico)')
axes[1].set_ylabel('Label')
axes[1].set_xlabel('Tempo (s)')
axes[1].set_ylim(-0.1, 1.1)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Modelo de Detecção de Picos (1D CNN)

Usamos uma CNN 1D simples que recebe a janela PPG e prediz a probabilidade
de cada ponto ser um pico.

> **Arquitetura:** Input → Conv1D → BatchNorm → ReLU → ... → Sigmoid

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Verificar se GPU disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Usando dispositivo: {device}")


class PeakDetectorCNN(nn.Module):
    """
    CNN 1D para detecção de picos em sinais PPG.
    
    Arquitetura U-Net simplificada: encoder-decoder com skip connections.
    """
    def __init__(self, input_size=250):
        super().__init__()
        
        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=7, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Conv1d(32, 32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        
        self.enc2 = nn.Sequential(
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        
        self.enc3 = nn.Sequential(
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        # Decoder
        self.dec2 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv1d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        
        self.dec1 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv1d(64 + 64, 32, kernel_size=3, padding=1),  # Skip connection
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        
        # Output
        self.output = nn.Sequential(
            nn.Conv1d(32 + 32, 16, kernel_size=3, padding=1),  # Skip connection
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Conv1d(16, 1, kernel_size=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # x: (batch, 1, window_size)
        
        # Encoder
        e1 = self.enc1(x)           # (batch, 32, window_size)
        e2 = self.enc2(e1)          # (batch, 64, window_size/2)
        e3 = self.enc3(e2)          # (batch, 128, window_size/4)
        
        # Decoder com skip connections
        d2 = self.dec2(e3)          # (batch, 64, window_size/2)
        d2 = torch.cat([d2, e2], dim=1)  # (batch, 128, window_size/2)
        
        d1 = self.dec1(d2)          # (batch, 32, window_size)
        d1 = torch.cat([d1, e1], dim=1)  # (batch, 64, window_size)
        
        out = self.output(d1)       # (batch, 1, window_size)
        
        return out.squeeze(1)       # (batch, window_size)


# Criar modelo
model = PeakDetectorCNN(WINDOW_SIZE).to(device)
print(f"🏗️ Modelo criado com {sum(p.numel() for p in model.parameters()):,} parâmetros")

## 6. Treinamento

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=30, batch_size=32, lr=0.001):
    """
    Treina o modelo de detecção de picos.
    """
    # Converter para tensors PyTorch
    X_train_t = torch.FloatTensor(X_train).unsqueeze(1)  # Adicionar canal
    y_train_t = torch.FloatTensor(y_train)
    X_val_t = torch.FloatTensor(X_val).unsqueeze(1)
    y_val_t = torch.FloatTensor(y_val)
    
    # DataLoaders
    train_dataset = TensorDataset(X_train_t, y_train_t)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Loss e optimizer
    criterion = nn.BCELoss()  # Binary Cross Entropy
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    # Histórico
    history = {'train_loss': [], 'val_loss': []}
    
    for epoch in range(epochs):
        model.train()
        train_losses = []
        
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
        
        # Validação
        model.eval()
        with torch.no_grad():
            X_val_t_dev = X_val_t.to(device)
            y_val_t_dev = y_val_t.to(device)
            y_val_pred = model(X_val_t_dev)
            val_loss = criterion(y_val_pred, y_val_t_dev).item()
        
        train_loss = np.mean(train_losses)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        
        scheduler.step(val_loss)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1:3d}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    return history


# Treinar
print("🚀 Iniciando treinamento...")
history = train_model(model, X_train, y_train, X_val, y_val, epochs=30)

### Curvas de Aprendizado

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history['train_loss'], label='Treino')
plt.plot(history['val_loss'], label='Validação')
plt.xlabel('Época')
plt.ylabel('Loss (BCE)')
plt.title('📈 Curvas de Aprendizado')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. Avaliação: Predição vs Ground Truth

In [ ]:
# Predição no conjunto de validação
model.eval()
with torch.no_grad():
    X_val_t = torch.FloatTensor(X_val).unsqueeze(1).to(device)
    y_pred = model(X_val_t).cpu().numpy()

# Visualizar algumas predições
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

for i, ax in enumerate(axes):
    idx = i * 2  # Amostras espaçadas
    
    t_window = np.arange(WINDOW_SIZE) / TARGET_FS
    
    # PPG
    ax.plot(t_window, X_val[idx], 'purple', alpha=0.7, label='PPG')
    
    # Ground truth
    ax.fill_between(t_window, y_val[idx] * X_val[idx].max() * 0.3, 
                    alpha=0.3, color='green', label='Ground Truth')
    
    # Predição
    ax.fill_between(t_window, y_pred[idx] * X_val[idx].max() * 0.3, 
                    alpha=0.3, color='red', label='Predição')
    
    ax.set_ylabel('PPG')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

axes[0].set_title('🎯 Comparação: Predição vs Ground Truth')
axes[-1].set_xlabel('Tempo (s)')
plt.tight_layout()
plt.show()

## 8. Usando nos Seus Dados (ESP32)

Agora vamos carregar um sinal do seu ESP32 e rodar o modelo treinado.

> **Importante:** Seus dados estão a 757Hz, precisamos decimar para 125Hz.

In [ ]:
def load_esp32_data(filepath, target_fs=125):
    """
    Carrega dados do ESP32 e decima para a taxa alvo.
    """
    import csv
    
    # Tentar carregar CSV
    try:
        data = np.loadtxt(filepath, delimiter=',', skiprows=1)
        ppg = data[:, 0]  # Assumindo primeira coluna é IR/PPG
        original_fs = 757  # Seu ESP32
    except:
        print(f"⚠️ Não foi possível carregar {filepath}")
        print("   Gerando dado sintético para demonstração...")
        ppg, _, original_fs = generate_synthetic_data(30, 757)
    
    # Decimar
    if original_fs != target_fs:
        num_samples = int(len(ppg) * target_fs / original_fs)
        ppg_resampled = resample(ppg, num_samples)
        print(f"📉 Decimado de {original_fs}Hz para {target_fs}Hz")
    else:
        ppg_resampled = ppg
    
    return ppg_resampled, target_fs


def predict_peaks(model, ppg, fs, window_size=250, threshold=0.5):
    """
    Usa o modelo para detectar picos no sinal PPG.
    
    Returns:
        peak_indices: Índices dos picos detectados
    """
    # Normalizar
    ppg_norm = (ppg - np.mean(ppg)) / np.std(ppg)
    
    # Criar janelas sobrepostas
    stride = window_size // 4  # 75% overlap para suavização
    predictions = np.zeros(len(ppg))
    counts = np.zeros(len(ppg))
    
    model.eval()
    with torch.no_grad():
        for start in range(0, len(ppg) - window_size, stride):
            window = ppg_norm[start:start + window_size]
            window_t = torch.FloatTensor(window).unsqueeze(0).unsqueeze(0).to(device)
            
            pred = model(window_t).cpu().numpy().squeeze()
            
            predictions[start:start + window_size] += pred
            counts[start:start + window_size] += 1
    
    # Média das predições sobrepostas
    predictions = predictions / np.maximum(counts, 1)
    
    # Encontrar picos na probabilidade predita
    peaks, _ = find_peaks(predictions, height=threshold, distance=int(0.4 * fs))
    
    return peaks, predictions


# Carregar dados do ESP32 (ou sintético se não disponível)
print("📂 Carregando dados do ESP32...")
esp32_ppg, esp32_fs = load_esp32_data(
    '/home/douglas/Documentos/Projects/PPG/pulse-analytics/analytics/datasets/sample.csv',
    target_fs=TARGET_FS
)

# Predizer picos
print("🔮 Detectando picos...")
detected_peaks, peak_probs = predict_peaks(model, esp32_ppg, esp32_fs)

print(f"✅ Detectados {len(detected_peaks)} picos")

# Calcular HR
if len(detected_peaks) > 1:
    rr = np.diff(detected_peaks) / esp32_fs * 1000  # ms
    hr = 60000 / np.mean(rr)
    print(f"❤️ HR estimado: {hr:.1f} BPM")

### Visualização da Predição nos Seus Dados

In [ ]:
# Plotar resultado
t = np.arange(len(esp32_ppg)) / esp32_fs

fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)

# PPG com picos detectados
axes[0].plot(t, esp32_ppg, 'purple', linewidth=0.8)
axes[0].scatter(t[detected_peaks], esp32_ppg[detected_peaks], 
                c='red', s=80, zorder=5, label='Picos Detectados')
axes[0].set_ylabel('PPG')
axes[0].set_title('🎯 Detecção de Picos no Seu Sinal (ESP32)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Probabilidade de pico
axes[1].plot(t, peak_probs, 'orange', linewidth=0.8)
axes[1].axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Threshold')
axes[1].set_ylabel('P(pico)')
axes[1].set_xlabel('Tempo (s)')
axes[1].set_ylim(-0.05, 1.05)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Salvando o Modelo

Salve o modelo treinado para usar posteriormente.

In [ ]:
# Salvar modelo
model_path = '/home/douglas/Documentos/Projects/PPG/pulse-analytics/analytics/peak_detector_model.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'window_size': WINDOW_SIZE,
    'target_fs': TARGET_FS
}, model_path)

print(f"💾 Modelo salvo em: {model_path}")

## 🎓 Resumo do Pipeline

1. **MIMIC-II** fornece o par PPG + ECG sincronizado
2. Detectamos picos R do **ECG** (algoritmo clássico, ~99% acurácia)
3. Transferimos para o **PPG** (+200ms de PTT)
4. Criamos **janelas rotuladas** (X = PPG, y = onde tem pico)
5. Treinamos uma **CNN 1D** para prever pico
6. **Fine-tuning**: usar seus dados para adaptar ao ruído do seu sensor
7. **Inferência**: o modelo prediz picos no seu sinal novo

---

### Próximos Passos
- [ ] Baixar registros reais do MIMIC-II (PhysioNet)
- [ ] Coletar mais dados seus para fine-tuning
- [ ] Testar arquitetura Performer (Attention) no lugar da CNN
- [ ] Exportar para TensorFlow Lite para rodar no ESP32